In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import cv2

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [2]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

master_path = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
master = pd.read_csv(master_path)
master.MRC_ID_DI[master.MRC_ID_DI > 0 ] = 1

In [ ]:
"""
master_0 = master[master['MRC_ID_DI'] == 0].sample(frac=1)
master_1 = master[master['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(master_0) if len(master_0) < len(master_1) else len(master_1)

master = pd.concat([master_0.head(sample_size), master_1.head(sample_size)]).sample(frac=1)
"""

In [4]:
from PIL import Image
img_path = os.path.join(os.getcwd(), 'image2')

y = []
X = []

for i in range(len(master)):
    path = os.path.join(img_path, str(master.iloc[i, 0]) +'.png')
    label = master.iloc[i, 1]
    img = Image.open(path)
    data = np.asarray(img)
    X.append(data)
    y.append(label)
    
    #for ang in range(-20, 20, 5):
    #    if ang != 0:
    #        img2 = img.rotate(ang)
    #        data = np.asarray(img2)
    #        X.append(data)
    #        y.append(label)
    #    
    #        img2 = img2.transpose(Image.FLIP_LEFT_RIGHT)
    #        data = np.asarray(img2)
    #        X.append(data)
    #        y.append(label)
        
X = np.array(X)       
y = np.array(y)
y = tf.keras.utils.to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify = y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1, stratify = y_train)

In [ ]:
"""
img_path = os.path.join(os.getcwd(), 'image')

y = []
X_paths = []

for i in range(len(master)):
    X_paths.append( os.path.join(img_path, str(master.iloc[i, 0]) +'.png') )
    y.append(master.iloc[i, 1])

y = np.array(y)
y = tf.keras.utils.to_categorical(y)

X_train_paths, X_test_paths, y_train, y_test = train_test_split(X_paths, y, test_size=0.1, random_state=1)

X_train_paths, X_val_paths, y_train, y_val = train_test_split(X_train_paths, y_train, test_size=0.1, random_state=1)

X_train = []
for file_path in X_train_paths:
    #read image
    img = cv2.imread(file_path)
    X_train.append(img)

X_train = np.array(X_train)

X_val = []
for file_path in X_val_paths:
    #read image
    img = cv2.imread(file_path)
    X_val.append(img)

X_val = np.array(X_val)

X_test = []
for file_path in X_test_paths:
    #read image
    img = cv2.imread(file_path)
    X_test.append(img)

X_test = np.array(X_test)
"""

In [5]:
img_rows, img_cols, img_channel = 224, 224, 3

base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [7]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dropout(rate = 0.8))
#add_model.add(tf.keras.layers.Dense(units=8, activation=tf.nn.relu))
add_model.add(tf.keras.layers.Dense(units=2, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0000001),
              metrics=[tf.keras.metrics.AUC()])#['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
batch_size = 2
epochs = 2000

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
train_datagen.fit(X_train)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
val_datagen.fit(X_val)


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)

history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size),
    callbacks=[callback]
)

# callbacks=[tf.keras.callbacks.ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 4099 steps, validate for 456 steps
Epoch 1/2000
4099/4099 [==============================] - 123s 30ms/step - loss: 1.9800 - auc_1: 0.5498 - val_loss: 0.8366 - val_auc_1: 0.6371
Epoch 2/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 1.5914 - auc_1: 0.6603 - val_loss: 0.7708 - val_auc_1: 0.7720
Epoch 3/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 1.4813 - auc_1: 0.7088 - val_loss: 0.7895 - val_auc_1: 0.7966
Epoch 4/2000
4099/4099 [==============================] - 115s 28ms/step - loss: 1.3870 - auc_1: 0.7439 - val_loss: 0.8121 - val_auc_1: 0.8050
Epoch 5/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 1.3444 - auc_1: 0.7607 - val_loss: 0.8219 - val_auc_1: 0.8101
Epoch 6/2000
4099/4099 [==============================] - 115s 28ms/step - loss: 1.3573 - auc_1: 0.7664 - val_loss

4099/4099 [==============================] - 113s 28ms/step - loss: 0.8432 - auc_1: 0.8590 - val_loss: 0.7060 - val_auc_1: 0.8589
Epoch 54/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 0.8622 - auc_1: 0.8572 - val_loss: 0.7041 - val_auc_1: 0.8592
Epoch 55/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 0.8858 - auc_1: 0.8542 - val_loss: 0.7009 - val_auc_1: 0.8615
Epoch 56/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 0.8170 - auc_1: 0.8656 - val_loss: 0.7033 - val_auc_1: 0.8605
Epoch 57/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 0.8640 - auc_1: 0.8571 - val_loss: 0.7067 - val_auc_1: 0.8600
Epoch 58/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 0.8359 - auc_1: 0.8605 - val_loss: 0.7094 - val_auc_1: 0.8601
Epoch 59/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 0.8162 - auc_1: 0.8656 - val_loss: 0.7026 - val_auc_1: 0.8612
Epoch 

4099/4099 [==============================] - 115s 28ms/step - loss: 0.6111 - auc_1: 0.9047 - val_loss: 0.6989 - val_auc_1: 0.8685
Epoch 110/2000
4099/4099 [==============================] - 113s 28ms/step - loss: 0.5712 - auc_1: 0.9107 - val_loss: 0.6939 - val_auc_1: 0.8683
Epoch 111/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 0.5794 - auc_1: 0.9096 - val_loss: 0.6954 - val_auc_1: 0.8690
Epoch 112/2000
4099/4099 [==============================] - 114s 28ms/step - loss: 0.5822 - auc_1: 0.9105 - val_loss: 0.6991 - val_auc_1: 0.8688
Epoch 113/2000
4099/4099 [==============================] - 115s 28ms/step - loss: 0.5815 - auc_1: 0.9092 - val_loss: 0.6997 - val_auc_1: 0.8689
Epoch 114/2000
4099/4099 [==============================] - 116s 28ms/step - loss: 0.5731 - auc_1: 0.9120 - val_loss: 0.6968 - val_auc_1: 0.8691
Epoch 115/2000
4099/4099 [==============================] - 115s 28ms/step - loss: 0.5619 - auc_1: 0.9138 - val_loss: 0.6972 - val_auc_1: 0.8703


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['auc_1'])
plt.plot(history.history['val_auc_1'])

plt.title('model auc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1/255.)#,
        #rotation_range=30, 
        #width_shift_range=0.1,
        #height_shift_range=0.1, 
        #horizontal_flip=True)
test_datagen.fit(X_test)

score = model.evaluate(test_datagen.flow(X_test, y_test, batch_size=batch_size), verbose=1)

In [ ]:
preds = model.predict(test_datagen.flow(X_test, batch_size=batch_size), verbose=1)

In [ ]:
def LIFT(preds, y_test): # >=2.5
    
    c = np.amax(preds, axis=1)
    b = np.argsort(-c)[:len(c)//5]

    lift_preds = preds[b]
    
    lift_preds_flat = np.argmax(lift_preds, axis=1)
    y_test_flat = np.argmax(y_test, axis=1)
    
        
    lift_y_true = lift_preds_flat[lift_preds_flat == 1]
    y_true = y_test_flat[y_test_flat == 1]
    
    print('LIFT Accuracy: ',  (len(lift_y_true)/len(lift_preds_flat))/(len(y_true)/len(y_test_flat)))

In [ ]:
lift_value = LIFT(preds, y_test)

In [ ]:
m = tf.keras.metrics.AUC()
m.update_state(y_test, preds)
auroc_value = m.result().numpy()
print(auroc_value)

In [ ]:
if lift_value:
    final_score = (lift_value/5)*0.7 + (auroc_value)*0.3
else:
    final_score = (auroc_value)*0.3
    
print(final_score)